<div align=center>
		
<p></p>
<p></p>
<font size=5>
In the Name of God
<font/>
<p></p>
 <br/>
    <br/>
    <br/>
<font color=#FF7500>
Sharif University of Technology - Departmenet of Computer Engineering
</font>
<p></p>
<font color=blue>
Artifical Intelligence - Dr. Mohammad Hossein Rohban
</font>
<br/>
<br/>
Fall 2021

</div>

<hr/>
		<div align=center>
		    <font color=red size=6>
			    <br />
Practical Assignment 5-Q1
            	<br/>
			</font>
    <br/>
    <br/>
<font size=4>
			<br/><br/>
Deadline:  Bahman 17th
                <br/><b>
              Cheating is Strongly Prohibited
                </b><br/><br/>
                <font color=red>
Please run all the cells.
     </font>
</font>
                <br/>
    </div>

# Personal Data

In [1]:
# Set your student number
student_number = 98109729
Name = 'Paniz'
Last_Name = 'Halvachi'

# Rules
- You are not allowed to use provided codes that can be found on the internet. 
- If you want to use a library which is not already imported, you must ask a question on Quera to get the permission of using that.
- Do not hesitate to ask questions on Quera, if you have any.
- This assignment is due Bahman 17th 23:59:59. you can use up to 1 grace day for this assignment and the hard deadline is Bahman 18th 23:59:59.

# Q1 (30Points)

## Mountain Car

The OpenAI Gym library includes a set of Python Reinforcement Learning environments. In this question, we examine the Mountain Car environment of this collection. Mountain Car is a Reinforcement Learning task that aims to learn the policy of climbing a steep hill and reaching the flag-marked goal. Also, the car engine is not powerful enough to climb straight up the hill on the right. Therefore, it must gain enough acceleration by climbing the hill on the left.


In this case, the state of the car is determined by an array containing its position and speed.

| Num | Observation | Min | Max|
| --- | --- | --- | ---|
| 0 | Position | -1.3 | 0.5 |
| 1 | Velocity | -0.07 | 0.07|

The intelligent agent is allowed to perform three movements of right push, no push and left push in each step. The agent's move will be given to the environment and the environment returns the next state along with the movement reward. For each step that the car does not reach the target, a cost of -1 is considered. Use Q-learning to find the optimal policy in each case. To do this, you must complete the functions below.

** Note that you will receive the full score of this question only if you can achieve a score of -115 with your implementation.

<font size=4>
Author: Atoosa Chegini
			<br/>
                <font color=red>
Please run all the cells.
     </font>
</font>
                <br/>
    </div>

In [2]:
import numpy as np
import random
import gym

In [3]:
env = gym.make("MountainCar-v0")
# initialization (parameters such as learning rate, gamma, number of states, number of actions, and q_table)
alpha = 0.1
gamma = 0.99
number_of_states = 50
max_loops = 200000
episodes = 60000
number_of_actions = 3
epsilon = 0.5
rate_epsilon = epsilon/(episodes*2)
q_table = np.zeros((number_of_states,number_of_states, number_of_actions))
# You may change the inputs of any function as you desire.

Next cell wants you supplement two functions. First for transforming the continuous space into discrete one (in order to make using q_table feasible), second for updating q_values based on the last action done by agent.

In [4]:
# This is just one example of a discretization function. You can change it as you want.
def discretize_state(x, minn, step):
    return int((x - minn) / step)

def env_state_to_Q_state(state):
    [position, velocity] = state
    pos_low, pos_high = -1.2, 0.6
    vel_low, vel_high =  -0.07, 0.07
    # Complete this function!
    Q_state = np.zeros(2)
    pos_step = (pos_high-pos_low)/number_of_states
    vel_step = (vel_high-vel_low)/number_of_states
    # Discretize position
    ds = discretize_state(position,pos_low,pos_step)
    if ds >= number_of_states:
        Q_state[0]= number_of_states - 1 
    elif ds < 0:
        Q_state[0] = 0
    else:
        Q_state[0] = ds
    ds_vel = discretize_state(velocity,vel_low,vel_step)
    if ds_vel >= number_of_states:
        Q_state[1]= number_of_states - 1
    elif ds_vel < 0:
        Q_state[1] = 0
    else:
        Q_state[1] = ds_vel  
    return tuple(Q_state.astype(np.int))

def update_q(current_state, next_state, action, reward,epsilon):
    best_future_q = np.max(q_table[next_state[0],next_state[1]])
    # Complete this function!
    sample = reward + gamma * best_future_q
    q_table[current_state[0],current_state[1],action] = (1-alpha) * q_table[current_state[0],current_state[1],action] + alpha * sample
    next_action = get_action_considering_epsilon_greedy(next_state,epsilon)
    return next_action

def get_action_considering_epsilon_greedy(next_state,epsilon):
    p = np.random.random()  #Return random floats in the half-open interval [0.0, 1.0)
    if p < epsilon:
        #action = env.action_space.sample() ############ not sure
        action = np.random.choice(range(number_of_actions))
    else:
        action = np.argmax(q_table[next_state[0],next_state[1]])
    return action     




At the following cell, the ends of two functions are getting current action based on the policy and defining the training process respectively.

In [5]:
# You may change the inputs of any function as you desire.
def get_action(current_state):
    return np.argmax(q_table[current_state])


def q_learning():
    epsilon_temp = epsilon
    for j in range(episodes):
        done = False
        total_reward = 0
        state = env.reset()
        state = env_state_to_Q_state(state)
        action = get_action_considering_epsilon_greedy(state,epsilon_temp)
        #for t in range(max_loops):
        while not done:
            # Get next state and reward
            next_state, reward, done, dummy = env.step(action) 
            next_state = env_state_to_Q_state(next_state)
            action = update_q(state,next_state,action,reward,epsilon_temp)
            total_reward += reward
            state = next_state
            if done:
                state = env.reset()
                break
        epsilon_temp = epsilon_temp - rate_epsilon    
        if j%100 == 0:
            print("\r current episode ",j,"reward: ",total_reward,"epsilon: ",epsilon_temp,end=" ")
    pass
    # Complete this funciton!


def save_policy():
    policy = np.argmax(q_table, axis=2)
    np.save('policy.npy', policy)

In [6]:
q_learning()
save_policy()

C:\Users\ASUS\AppData\Local\Temp/ipykernel_8276/4145126848.py:28: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return tuple(Q_state.astype(np.int))


 current episode  59900 reward:  -191.0 epsilon:  0.2504124999983649   55700 reward:  -189.0 epsilon:  0.26791249999847955 

In [7]:
# Attention: don't change this function. we will use this to grade your policy which you will hand in with policy.npy
# btw you can use it to see how you are performing. Uncomment two lines which are commented to be able to see what is happening visually.
def score():
    policy, scores = np.load("policy.npy"), []
    print(policy)
    for episode in range(1000):
        print(f"******Episode {episode}")
        state, score, done, step = env_state_to_Q_state(env.reset()), 0, False, 0
        while not done:
            # time.sleep(0.04)
            action = policy[state]
            state, reward, done, _ = env.step(action)
            state = env_state_to_Q_state(state)
            step += 1
            score += int(reward)
            # env.render()
        print(f"Score:{score}")
        scores.append(score)
    print(f"Average score over 1000 run : {np.array(scores).mean()}")

score()

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
******Episode 0
Score:-85
******Episode 1
Score:-110
******Episode 2
Score:-88
******Episode 3
Score:-109
******Episode 4
Score:-108
******Episode 5
Score:-109
******Episode 6
Score:-85
******Episode 7
Score:-109
******Episode 8
Score:-108
******Episode 9
Score:-109
******Episode 10
Score:-85
******Episode 11
Score:-108
******Episode 12
Score:-85
******Episode 13
Score:-85
******Episode 14
Score:-85
******Episode 15
Score:-109
******Episode 16
Score:-108
******Episode 17
Score:-106
******Episode 18
Score:-92
******Episode 19
Score:-110
******Episode 20
Score:-106
******Episode 21
Score:-108
******Episode 22
Score:-86
******Episode 23
Score:-108

C:\Users\ASUS\AppData\Local\Temp/ipykernel_8276/4145126848.py:28: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return tuple(Q_state.astype(np.int))



******Episode 24
Score:-86
******Episode 25
Score:-86
******Episode 26
Score:-88
******Episode 27
Score:-111
******Episode 28
Score:-107
******Episode 29
Score:-111
******Episode 30
Score:-109
******Episode 31
Score:-85
******Episode 32
Score:-108
******Episode 33
Score:-109
******Episode 34
Score:-111
******Episode 35
Score:-88
******Episode 36
Score:-112
******Episode 37
Score:-108
******Episode 38
Score:-107
******Episode 39
Score:-166
******Episode 40
Score:-109
******Episode 41
Score:-109
******Episode 42
Score:-108
******Episode 43
Score:-111
******Episode 44
Score:-108
******Episode 45
Score:-108
******Episode 46
Score:-94
******Episode 47
Score:-109
******Episode 48
Score:-109
******Episode 49
Score:-111
******Episode 50
Score:-109
******Episode 51
Score:-108
******Episode 52
Score:-94
******Episode 53
Score:-109
******Episode 54
Score:-108
******Episode 55
Score:-111
******Episode 56
Score:-86
******Episode 57
Score:-107
******Episode 58
Score:-107
******Episode 59
Score:-108